In [6]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from tqdm import tqdm
#from lifelines impo

In [3]:
lab_test = pd.read_csv('~/Research/PHD/project/Hua Zheng/previous code/cleaned_full_No_CVD_EHR_medical_forward_backward_EMA_filling.csv', delimiter=',')
#lab_test_with_ms = pd.read_csv('~/Research/PHD/project/Hua Zheng/previous code/cleaned_full_No_CVD_EHR_medical.csv', delimiter=',')
param_class = pd.read_csv('~/Research/PHD/project/Hua Zheng/previous code/cleaned_EHR_treatment_param.csv', delimiter=',')
thera_class = pd.read_csv('~/Research/PHD/project/Hua Zheng/previous code/cleaned_EHR_treatment_thera.csv', delimiter=',')


In [4]:
lab_test = lab_test.sort_values(by=['study_id', 'ENCOUNTER_DT_RAN']).merge(param_class,on=['study_id', 'ENCOUNTER_DT_RAN'])


In [7]:
import time

data = []

demo_feature_list = ['study_id','ENCOUNTER_DT_RAN','age','smoke','SEX','RACE']
interest_feature_list = [
 'CREATININE',
 'HEMOGLOBIN A1C',
 'CHOLESTEROL, TOTAL',
 'HDL CHOLESTEROL',
 'TRIGLYCERIDES',
 'EGFR MDRD AFRICAN AMERICAN',
 'LDL CHOLESTEROL',
 'EGFR MDRD NON AFRICAN AMERICAN',
 #'CREATININE, URINE',
 'BP_SYSTOLIC','BP_DIASTOLIC',
 'BMI']

target_cloumn_names = param_class.columns[4:]
agg_max_features = {ftr:'max' for ftr in interest_feature_list} 
agg_min_features = {ftr:'min' for ftr in interest_feature_list} 
agg_max_targets = {targ:'max' for targ in target_cloumn_names} 

# engineer features
features_column_names = [ftr.lower().replace(' ','_') for ftr in demo_feature_list] + \
[ftr.lower().replace(' ','_') for ftr in interest_feature_list] + \
[ftr.lower().replace(' ','_')+'_max' for ftr in interest_feature_list] + \
[ftr.lower().replace(' ','_')+'_min' for ftr in interest_feature_list] + \
[tag.lower().replace(' ','_')+'_hist' for tag in target_cloumn_names] + \
[tag.lower().replace(' ','_')+'_cur' for tag in target_cloumn_names] + \
[tag.lower().replace(' ','_') for tag in target_cloumn_names]

#cnt= 0
#timestamp = time.time()
for patient, d in tqdm(lab_test.groupby(['study_id'])):
    cur_half_year_idx = 0
    total_num_visits = d.shape[0]
    for visit_idx in range(d.shape[0]):
        while d.iloc[visit_idx,0] - d.iloc[cur_half_year_idx,0] >= 366/2: 
            cur_half_year_idx += 1
        
        temp_past_half_year = d.iloc[cur_half_year_idx:visit_idx]
        cur_visit_lab_max = temp_past_half_year.agg(agg_max_features).tolist()
        cur_visit_lab_min = temp_past_half_year.agg(agg_min_features).tolist()
        demo_features = d.iloc[visit_idx][demo_feature_list].tolist()
        cur_visit_lab = d.iloc[visit_idx][interest_feature_list].tolist()
        treatment_history = temp_past_half_year[target_cloumn_names].agg(agg_max_targets).tolist()
        cur_treatment = d.iloc[visit_idx-1][target_cloumn_names].tolist() if visit_idx > 0 else [0]*len(target_cloumn_names)
        treatment = d.iloc[visit_idx][target_cloumn_names].tolist()
        #print(demo_features+cur_visit_lab+cur_visit_lab_max+cur_visit_lab_min)
        data.append(demo_features+cur_visit_lab+cur_visit_lab_max+cur_visit_lab_min+treatment_history+cur_treatment+treatment)
            
data
featurized_data = pd.DataFrame(data, columns=features_column_names)
#featurized_data

100%|██████████| 22952/22952 [6:09:56<00:00,  1.03it/s]   


In [8]:
data
featurized_data = pd.DataFrame(data, columns=features_column_names)
#featurized_data
featurized_data.to_csv('~/Research/PHD/project/Hua Zheng/previous code/cleaned_EHR_treatment_param_lab_test_final.csv', index=False)